In [41]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
import pandas as pd

we_eat_client = MongoClient()
we_eat_db = we_eat_client['we_eat']
website_collection = we_eat_db['websites']
more_business_info_collection = we_eat_db['more_business_info']

In [2]:
website_collection.count_documents({})

657

In [3]:
a = website_collection.find()[:3]

In [4]:
for website in a:
    html = website['html']
    soup = BeautifulSoup(html, 'html.parser')
    bizinfo = soup.select('div.short-def-list dl')
    data = get_biz_info(bizinfo)
    

NameError: name 'get_biz_info' is not defined

In [70]:
website = website_collection.find()[2:3]

In [71]:
html = website[0]['html']

In [72]:
soup = BeautifulSoup(html, 'html.parser')

In [73]:
bizinfo = soup.select('div.short-def-list dl')

In [76]:
bizinfo[0].find_all('dt')

[<dt class="attribute-key">
                             Liked by Vegans
                         </dt>]

In [11]:
len(bizinfo)

1

In [77]:
get_biz_info(bizinfo)

{'Accepts Apple Pay': 'No',
 'Accepts Bitcoin': 'No',
 'Accepts Credit Cards': 'Yes',
 'Accepts Google Pay': 'No',
 'Alcohol': 'Full Bar',
 'Ambience': 'Casual, Trendy, Classy',
 'Attire': 'Casual',
 'Bike Parking': 'Yes',
 'Caters': 'Yes',
 'Delivery': 'Yes',
 'Dogs Allowed': 'No',
 'Gender Neutral Restrooms': 'Yes',
 'Good For': 'Lunch, Dinner',
 'Good for Groups': 'Yes',
 'Good for Kids': 'Yes',
 'Has TV': 'No',
 'Liked by Vegans': 'Yes',
 'Liked by Vegetarians': 'Yes',
 'Noise Level': 'Average',
 'Outdoor Seating': 'Yes',
 'Parking': 'Street',
 'Take-out': 'Yes',
 'Takes Reservations': 'Yes',
 'Waiter Service': 'Yes',
 'Wheelchair Accessible': 'Yes',
 'Wi-Fi': 'Free'}

In [81]:
def parse_bizinfo(bizinfos, alias):
    biz = {}
    biz['alias'] = alias
    for bizinfo in bizinfos:
        dt = bizinfo.select_one('dt')
        cat_name = dt.text.strip()
        dd = bizinfo.select_one('dd')
        response = dd.text.strip()
        biz[cat_name] = response
    return biz

In [79]:
def save_bizinfo(bizinfo_data):
    more_business_info_collection.delete_many({
        'alias' : bizinfo_data['alias']
    })
    more_business_info_collection.insert_one(bizinfo_data)

In [80]:
def collect_bizinfo(website_collection):
    for website in website_collection.find():
        url = website['url']
        alias = url.rpartition('/')[2]
        html = website['html']
        soup = BeautifulSoup(html, 'html.parser')
        bizinfos = soup.select('div.short-def-list dl')
        print(alias)
        data = parse_bizinfo(bizinfos, alias)
        save_bizinfo(data)

In [40]:
%pdb

Automatic pdb calling has been turned OFF


In [82]:
collect_bizinfo(website_collection)

amazon-go-seattle-5
the-pink-door-seattle-4
nirmals-seattle
biscuit-bitch-seattle-5
damn-the-weather-seattle
bad-bishop-seattle
pike-place-chowder-seattle
tsukushinbo-seattle
taylor-shellfish-oyster-bar-seattle-2
elliotts-oyster-house-seattle-2
dough-zone-dumpling-house-seattle-2
the-london-plane-seattle-3
a-hong-kong-kitchen-seattle
radiator-whiskey-seattle
sizzling-pot-king-seattle-2
one-pot-seattle-3
ummas-lunch-box-seattle
dead-line-seattle
tats-delicatessen-seattle
harbor-city-restaurant-seattle
shawn-o-donnells-american-grill-and-irish-pub-seattle-2
commissary-korean-kitchen-seattle
king-noodle-seattle
bibimbap-king-seattle
good-bar-seattle-3
casco-antiguo-seattle
tofully-seattle-2
brgr-bar-seattle-seattle
japonessa-sushi-cocina-seattle
metropolitan-grill-seattle
i5-pho-seattle-seattle-2
mikes-noodle-house-seattle
salumi-artisan-cured-meats-seattle-2
maneki-seattle
copal-seattle
tamarind-tree-seattle
the-hart-and-the-hunter-seattle-2
pier-54-seattle
chucks-hole-in-the-wall-bbq-se

olivers-lounge-seattle
bavarian-meats-seattle
purple-dot-cafe-seattle-2
los-agaves-seattle-4
poké-bar-seattle-9
hawks-nest-seattle
bacco-cafe-and-bistro-seattle-2
dunbar-room-seattle
delicatus-seattle-2
mariscos-mexico-seattle
delaurenti-food-and-wine-seattle-2
harborview-medical-center-cafeteria-seattle
206-burger-company-seattle-2
dog-in-the-park-seattle
evergreens-salad-seattle-4
pita-xpress-seattle-4
frolik-kitchen-cocktails-seattle-2
the-diller-room-seattle
the-bistro-at-courtyard-seattle
urara-seattle-2
90-karaoke-seattle-2
cafe-pho-seattle
mee-sum-pastry-seattle
seattle-deli-seattle
blarney-stone-pub-and-restaurant-seattle
fuel-seattle-2
owln-thistle-irish-pub-seattle
il-bistro-seattle
joes-grilled-gourmet-dogs-seattle
nyc-deli-market-seattle-2
thai-taste-restaurant-seattle
the-lodge-sports-grille-downtown-seattle
fortune-garden-seattle
4th-ave-espresso-bar-seattle-3
oriental-mart-seattle-5
thanh-son-tofu-seattle-3
jimmys-on-first-seattle
walla-walla-farms-seattle-3
cone-and-ste

kebab-seattle-2
espresso-caffe-dior-seattle
1st-floor-cafe-seattle
nachyo-average-seattle
la-creperie-by-ethan-stowell-seattle-3
taco-del-mar-seattle-7
kidd-valley-seattle-7
tummy-yummy-thai-seattle
mediterranean-cuisine-seattle
cocina-buena-food-truck-seattle
ricenroll-seattle-2
subway-seattle-12
restaurant-specialty-services-seattle
union-deli-la-puget-sound-plaza-building-seattle
measure-seattle
subway-seattle-3
u-bank-cafe-seattle
subway-seattle-31
hamburg-frites-seattle
subway-seattle-66
organic-to-go-seattle-4
chez-dave-seattle-2


In [95]:
more_business_info_collection.count_documents({})

2628

In [99]:
more_business_info_collection.find_one({'alias' : 'nachyo-average-seattle'})

{'Delivery': 'No',
 'Good for Kids': 'Yes',
 'Take-out': 'Yes',
 'Takes Reservations': 'No',
 '_id': ObjectId('5c0841f291d56fd487300588'),
 'alias': 'nachyo-average-seattle'}

In [105]:
more_business_info_collection.delete_many({'alias': {'$exists': False}})

In [106]:
biz_dicts = pd.DataFrame(list(more_business_info_collection.find()))

In [111]:
cols = list(biz_dicts.columns)

In [112]:
sorted_cols = cols[-2:] + cols[:-2]

In [114]:
biz_dicts = biz_dicts[sorted_cols]

In [115]:
biz_dicts.head()

,_id,alias,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Accepts Google Pay,Alcohol,Ambience,Attire,Best Nights,...,Offers Military Discount,Open to All,Outdoor Seating,Parking,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi
0,5c0840ff91d56fd48730030a,amazon-go-seattle-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
1,5c0840ff91d56fd48730030b,the-pink-door-seattle-4,No,NaN,Yes,No,Full Bar,Romantic,Casual,"Wed, Fri, Sat",...,NaN,NaN,Yes,"Garage, Street",NaN,No,Yes,Yes,No,Free
2,5c08410091d56fd48730030c,nirmals-seattle,No,No,Yes,No,Full Bar,"Casual, Trendy, Classy",Casual,NaN,...,NaN,NaN,Yes,Street,NaN,Yes,Yes,Yes,Yes,Free
3,5c08410091d56fd48730030d,biscuit-bitch-seattle-5,Yes,NaN,Yes,NaN,No,"Hipster, Casual, Trendy",Casual,NaN,...,NaN,NaN,No,Street,NaN,Yes,No,No,Yes,Free
4,5c08410191d56fd48730030e,damn-the-weather-seattle,No,NaN,Yes,No,Full Bar,"Hipster, Casual, Trendy",Casual,"Fri, Sat, Sun",...,NaN,NaN,Yes,Street,No,No,No,Yes,Yes,No
